<a href="https://colab.research.google.com/github/hajnayeb/Python-Visualization/blob/main/airline_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas dash
!pip install dash
!pip install dash_core_components
!pip install dash_html_components
!pip install jupyter-dash

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
from urllib.request import urlopen
import json

In [4]:
# Loading the json file of the US states:
with urlopen('https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json') as response:
    usstates = json.load(response)
# Reading the airline data:
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1", dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})
airline_data.rename(columns={'Unnamed: 0':'fn'}, inplace=True)  # renaming the flight number column

In [6]:
# Reading the state id numbers
sdata =  pd.read_csv('https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv', 
                            encoding = "ISO-8859-1", dtype= str)
sdata.drop('stname', axis=1,inplace=True)
sdata.columns= ['st','sn']
sdata['st'] = sdata['st'].str.strip() # removing the undesired space in the names
sdata.tail(2)

,st,sn
49,55,WI
50,56,WY


In [7]:
airline_data["Quarter"] = airline_data["Quarter"].astype(str)# Should be commented after the first run
# adding a column to show the state code:
for n in range(0,airline_data.shape[0]-1):
    b=airline_data.loc[n,'OriginCityName'].split(",")
    x=sdata.loc[sdata['sn'] == b[1]]
    if x.shape[0] == 0:
        airline_data.iat[n,2]='02'
    else:
        airline_data.iat[n,2]=x.iloc[0,0]
airline_data.rename(columns={'Quarter': 'State'}, inplace=True) # renaming the column title
airline_data.tail(3)

,fn,Year,State,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,...,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
26997,9055,2016,48,8,26,5,2016-08-26,AA,19805,AA,N3HBAA,2183,10423,1042302,30423,AUS,"Austin, TX",TX,48.0,Texas,74,12478,1247803,31703,JFK,"New York, NY",NY,36.0,New York,22,920,914.0,-6.0,0.0,0.0,-1.0,0900-0959,13.0,927.0,1356.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26998,84136,2009,08,8,8,6,2009-08-08,YV,20378,YV,N516LR,7194,11109,1110901,30189,COS,"Colorado Springs, CO",CO,8.0,Colorado,82,11292,1129202,30325,DEN,"Denver, CO",CO,8.0,Colorado,82,1139,1230.0,51.0,51.0,1.0,3.0,1100-1159,17.0,1247.0,1306.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26999,113029,1993,3,7,17,6,1993-07-17,DL,19790,DL,NaN,310,12992,1299201,32600,LIT,"Little Rock, AR",AR,5.0,Arkansas,71,11298,1129802,30194,DFW,"Dallas/Fort Worth, TX",TX,48.0,Texas,74,615,618.0,3.0,3.0,0.0,0.0,0600-0659,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()
# Defining the range of years for the dashboard:
yr=list(range(2005,2021))

In [15]:
def compute_info(airline_data, entered_year):
    # Select data
    df =  airline_data[airline_data['Year']==int(entered_year)]
    # Compute the averages of different types of delay
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    # Compute performace metrics
    f_sum = df.groupby(['Month','Reporting_Airline'])['fn'].count().reset_index()
    cancel_sum = df.groupby(['Month','Reporting_Airline'])['Cancelled'].sum().reset_index()
    time_sum = df.groupby(['State'])['AirTime'].sum().reset_index()
    dist_sum = df.groupby(['Month','Reporting_Airline'])['Distance'].sum().reset_index()
    arr_del_sum = df.groupby(['Month','Reporting_Airline'])['ArrDelay'].sum().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late, f_sum, cancel_sum, time_sum, dist_sum, arr_del_sum

In [17]:
# Building the layout of the dash app 
app.layout = html.Div(children=[ html.H1('US Domestic Airline Flights Performance', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 30}),
                  #Dropdown #1:                              
                                html.Label("Report type: ",style={'font-size':18,'font-weight':'bold'}),
                                dcc.Dropdown(id = 'input-type', options=[
                                {'label': 'Yearly airline performance report', 'value': 'YP'},
                                {'label': 'Yearly average flight delay statistics', 'value': 'YD'}] ,value='YP',
                            placeholder="Yearly airline performance report",
                            style = dict(
                            width = '50%',
                            display = 'inline-block',
                            verticalAlign = "middle" )),  
                                ####################
                  #Dropdown #2: 
                                html.Label("Choose Year: ",style={'font-size':18,'font-weight':'bold'}),
                                dcc.Dropdown(id = 'input-year', options=[{"label": i, "value": i} for i in yr], value='2015',
                            placeholder="2016",
                            style = dict(
                            width = '30%',
                            display = 'inline-block',
                            verticalAlign = "middle" )),
                                #####################
                                html.Br(),
                                html.Br(), 
                                html.Div(dcc.Graph(id='treemap-plot'), style={'width':'65%'}),
                                html.Div([
                                        html.Div(dcc.Graph(id='pie-plot')),
                                        html.Div(dcc.Graph(id='map-plot'))
                                ], style={'display': 'flex'}),
    
                                html.Div([
                                        html.Div(dcc.Graph(id='bar-plot')),
                                        html.Div(dcc.Graph(id='line-plot'))
                                ], style={'display': 'flex'}),
                                ])

In [19]:
# Callback decorator
@app.callback( [
               Output(component_id='treemap-plot', component_property='figure'),
               Output(component_id='pie-plot', component_property='figure'),
               Output(component_id='map-plot', component_property='figure'),
               Output(component_id='bar-plot', component_property='figure'),
               Output(component_id='line-plot', component_property='figure')
               ],
               [Input(component_id='input-type', component_property='value'),
                Input(component_id='input-year', component_property='value'),
               ])
# Computation to callback function and return graph
def get_graph(entered_type,entered_year):
    
    # Compute required information for creating graphs from the data using the defined function of airline data and studied year
    [avg_car, avg_weather, avg_NAS, avg_sec, avg_late, f_sum, cancel_sum, time_sum, dist_sum, arr_del_sum] = compute_info(airline_data, entered_year)
            
    if entered_type == 'YP':
        # Create graphs of the performance analysis    
        carrier_fig = px.treemap(f_sum, path=['Month','Reporting_Airline'], values='fn', title='Number of flights by airline')
        weather_fig = px.sunburst(dist_sum, path=['Month','Reporting_Airline'], values='Distance', title='Total travelled distance (kilometers) by airline')
        nas_fig = px.choropleth_mapbox(time_sum, geojson=usstates, locations='State', color='AirTime',
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=2.5, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'unemp':'unemployment rate'},
                           title='Total air time (minutes) by state')
        sec_fig = px.bar(cancel_sum, x="Month", y="Cancelled", color="Reporting_Airline", title="Total cancelled flights by airline")
        late_fig = px.line(arr_del_sum, x='Month', y='ArrDelay', color="Reporting_Airline", title='Total arrival delay (minutes) by airline')
    else :
            # Create graphs of the delay analysis
        carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay',color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline')
        weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay',color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
        nas_fig = px.line(avg_NAS, x='Month', y='NASDelay',color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
        sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay',color='Reporting_Airline', title='Average security delay time (minutes) by airline')
        late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay',color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')
            
    return[carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]


# Run the app
if __name__ == '__main__':
    app.run_server(mode="external", host="localhost", port=7652, debug=True)

Dash app running on:


<IPython.core.display.Javascript object>